Cookbook
--

A replication exercises of [This Link](http://pandas.pydata.org/pandas-docs/stable/cookbook.html).

In [111]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame, Panel, datetime
from datetime import timedelta

## Idioms

These are some neat pandas idioms.

if-then/if-then-else on one columns, and assignment to another one or more columns.

In [112]:
df = DataFrame({'AAA': [4, 5, 6, 7], 
                'BBB': [10, 20, 30, 40],
                'CCC': [100,50,-30,-50]})
df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [113]:
# if-then on one column
df.ix[df.AAA >= 5, 'BBB'] = -1; df

,AAA,BBB,CCC
0,4,10,100
1,5,-1,50
2,6,-1,-30
3,7,-1,-50


In [114]:
# if-then with assignment to 2 columns
df.ix[df.AAA >=5, ['BBB', 'CCC']] = 555; df

,AAA,BBB,CCC
0,4,10,100
1,5,555,555
2,6,555,555
3,7,555,555


In [115]:
# Or use pandas .where() after you've set up a mask
df_mask = DataFrame({'AAA': [True]*4, 
                     'BBB': [True]*4,
                     'CCC': [True, False]*2})
df.where(df_mask, -999)

,AAA,BBB,CCC
0,4,10,100
1,5,555,-999
2,6,555,555
3,7,555,-999


## Splitting

Split a frame with a boolean criterion.


In [116]:
df = DataFrame({'AAA': [4, 5, 6, 7], 
                'BBB': [10, 20, 30, 40],
                'CCC': [100,50,-30,-50]})
df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [117]:
df_low = df[df.AAA <=5]
df_high = df[df.AAA > 5]
print(df_low, '\n', df_high)

   AAA  BBB  CCC
0    4   10  100
1    5   20   50 
    AAA  BBB  CCC
2    6   30  -30
3    7   40  -50


## Building Criteria

Select with multi-column criteria.

In [118]:
df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [119]:
# AND (without assignment returns a Series)
new_series = df.loc[(df.BBB < 25) & (df.CCC > -40), 'AAA']
new_series

0    4
1    5
Name: AAA, dtype: int64

In [120]:
# OR (with assignment modifies the DataFrame)
df.loc[(df.BBB < 25) | (df.CCC > -40), 'AAA'] = 0.1
df

,AAA,BBB,CCC
0,0.1,10,100
1,0.1,20,50
2,0.1,30,-30
3,7.0,40,-50


#### Select rows with data closest to certain value using argsort.

In [121]:
df = DataFrame({'AAA': [4, 5, 6, 7], 
                'BBB': [10, 20, 30, 40],
                'CCC': [100,50,-30,-50]})
df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [122]:
a_value = 43.0
df.ix[(df.CCC-a_value).abs().argsort()]

,AAA,BBB,CCC
1,5,20,50
0,4,10,100
2,6,30,-30
3,7,40,-50


#### Dynamically reduce a list of criteria using a binary operators

In [123]:
crit1 = df.AAA <= 5.5
crit2 = df.BBB == 10.0
crit3 = df.CCC > -40.0
all_crit = crit1 & crit2 & crit3
df[all_crit]

,AAA,BBB,CCC
0,4,10,100


## Selections

#### DataFrames - Using both row labels and value conditionals

In [124]:
df[(df.AAA <= 6) & (df.index.isin([0, 2, 4]))]

,AAA,BBB,CCC
0,4,10,100
2,6,30,-30


#### Using inverse operator (~) to take the complement of a mask

In [125]:
df[~ ((df.AAA <= 6) & (df.index.isin([0, 2, 4])))]

,AAA,BBB,CCC
1,5,20,50
3,7,40,-50


#### Panels - Direct Assignment

In [126]:
rng = pd.date_range('2015-11-11', periods=100, freq='D')
data = np.random.randn(100, 4)
cols = ['A', 'B', 'C', 'D']
df1, df2, df3 = DataFrame(data, rng, cols), DataFrame(data, rng, cols), DataFrame(data, rng, cols)
data_panel = Panel({'df1': df1, 
                    'df2': df2,
                    'df3': df3})
data_panel

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 100 (major_axis) x 4 (minor_axis)
Items axis: df1 to df3
Major_axis axis: 2015-11-11 00:00:00 to 2016-02-18 00:00:00
Minor_axis axis: A to D

In [127]:
# Direct assignment
data_panel.loc[:, :, 'F'] = DataFrame(data, rng, cols)
data_panel

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 100 (major_axis) x 5 (minor_axis)
Items axis: df1 to df3
Major_axis axis: 2015-11-11 00:00:00 to 2016-02-18 00:00:00
Minor_axis axis: A to F

## New Columns

Efficiently and dynamically creating new columns using .applymap()

In [128]:
df = pd.DataFrame(
   ....:      {'AAA' : [1,2,1,3], 'BBB' : [1,1,2,2], 'CCC' : [2,1,3,1]}); df
   ....: 
df

,AAA,BBB,CCC
0,1,1,2
1,2,1,1
2,1,2,3
3,3,2,1


In [129]:
source_cols = df.columns
new_cols = [str(x) + '_cat' for x in source_cols]
categories = {1: 'Alpha',
              2: 'Beta',
              3: 'Charlie'}
df[new_cols] = df[source_cols].applymap(categories.get)
df

,AAA,BBB,CCC,AAA_cat,BBB_cat,CCC_cat
0,1,1,2,Alpha,Alpha,Beta
1,2,1,1,Beta,Alpha,Alpha
2,1,2,3,Alpha,Beta,Charlie
3,3,2,1,Charlie,Beta,Alpha


Keep other columns when using .min() using groupby

In [130]:
df = pd.DataFrame(
   ....:      {'AAA' : [1,1,1,2,2,2,3,3], 'BBB' : [2,1,3,4,5,1,2,3]}); df
   ....: 
df

,AAA,BBB
0,1,2
1,1,1
2,1,3
3,2,4
4,2,5
5,2,1
6,3,2
7,3,3


In [131]:
# Method 1: .idxmin() to get the index of the mins
df.loc[df.groupby('AAA')['BBB'].idxmin()]

,AAA,BBB
1,1,1
5,2,1
6,3,2


In [132]:
# Method 2: Sort then take first of each
df.sort(columns='BBB').groupby('AAA', as_index=False).first()

,AAA,BBB
0,1,1
1,2,1
2,3,2


## MultiIndexing


In [133]:
# Creating a multi-index from a labeled frame
df = pd.DataFrame({'row' : [0,1,2],
                       'One_X' : [1.1,1.1,1.1],
                       'One_Y' : [1.2,1.2,1.2],
                       'Two_X' : [1.11,1.11,1.11],
                       'Two_Y' : [1.22,1.22,1.22]})
df

,One_X,One_Y,Two_X,Two_Y,row
0,1.1,1.2,1.11,1.22,0
1,1.1,1.2,1.11,1.22,1
2,1.1,1.2,1.11,1.22,2


In [134]:
# As labelled index
df = df.set_index('row'); df

,One_X,One_Y,Two_X,Two_Y
row,,,,
0,1.1,1.2,1.11,1.22
1,1.1,1.2,1.11,1.22
2,1.1,1.2,1.11,1.22


In [135]:
# With heirachical columns
df.columns = pd.MultiIndex.from_tuples([
        tuple(c.split('_')) for c in df.columns
    ])
df

One        Two      
       X    Y     X     Y
row                      
0    1.1  1.2  1.11  1.22
1    1.1  1.2  1.11  1.22
2    1.1  1.2  1.11  1.22

In [140]:
# Now stack and reset
df = df.stack(0).reset_index(1); df

In [141]:
df.columns = ['sample', 'all_x', 'all_y']; df

,sample,all_x,all_y
row,,,
0,One,1.10,1.20
0,Two,1.11,1.22
1,One,1.10,1.20
1,Two,1.11,1.22
2,One,1.10,1.20
2,Two,1.11,1.22


#### Multi-Index Arithmetic - Performing arithmetic with a multi-index that needs broadcasting

In [143]:
cols = pd.MultiIndex.from_tuples([ (x,y) for x in ['A','B','C'] for y in ['O','I']])
df = pd.DataFrame(np.random.randn(2,6),index=['n','m'],columns=cols); df

A                   B                   C          
          O         I         O         I         O         I
n -0.787947 -0.433019  0.639489  0.582353  0.136021  0.178723
m -0.838121 -1.249015  1.604562  0.939099 -1.516880  1.685522

In [150]:
df = df.div(df['C'], level=1); df

A                   B            C   
          O         I         O         I  O  I
n -5.792843 -2.422854  4.701409  3.258416  1  1
m  0.552530 -0.741026 -1.057804  0.557156  1  1

## Missing Data

#### Fill forward a reversed timeseries

In [152]:
df = DataFrame(np.random.randn(6, 1), 
               index=pd.date_range('2015-11-11', periods=6, freq='B'),
               columns = list('A'))
df

,A
2015-11-11,0.530275
2015-11-12,-0.300791
2015-11-13,-0.877953
2015-11-16,-0.023220
2015-11-17,0.233134
2015-11-18,-1.675380


In [154]:
df.ix[3, 'A'] = np.nan
df

,A
2015-11-11,0.530275
2015-11-12,-0.300791
2015-11-13,-0.877953
2015-11-16,NaN
2015-11-17,0.233134
2015-11-18,-1.675380


In [156]:
df.reindex(df.index[::-1]).ffill()

,A
2015-11-18,-1.675380
2015-11-17,0.233134
2015-11-16,0.233134
2015-11-13,-0.877953
2015-11-12,-0.300791
2015-11-11,0.530275


## Grouping

#### Basic grouping with apply

Unlike .agg( ), .apply( )'s callable is passed a sub-DataFrame which gives you access to all the columns.

In [158]:
df = pd.DataFrame({'animal': 'cat dog cat fish dog cat cat'.split(),
   ....:                    'size': list('SSMMMLL'),
   ....:                    'weight': [8, 10, 11, 1, 20, 12, 12],
   ....:                    'adult' : [False] * 5 + [True] * 2}); df
df

,adult,animal,size,weight
0,False,cat,S,8
1,False,dog,S,10
2,False,cat,M,11
3,False,fish,M,1
4,False,dog,M,20
5,True,cat,L,12
6,True,cat,L,12


In [160]:
# List the size of the animals with the highest weight
df.groupby('animal').apply(lambda x: x['size'][x['weight'].idxmax()])

animal
cat     L
dog     M
fish    M
dtype: object

In [162]:
# Use .getgroup() to retrieve info of a DataFrame
gb = df.groupby(['animal'])
gb.get_group('cat')

,adult,animal,size,weight
0,False,cat,S,8
2,False,cat,M,11
5,True,cat,L,12
6,True,cat,L,12


#### Apply to different items in a group

In [166]:
def GrowUp(x):
    avg_weight = sum(x[x['size'] == 'S'].weight * 1.5)
    avg_weight += sum(x[x['size'] == 'M'].weight * 1.25)
    avg_weight += sum(x[x['size'] == 'L'].weight)
    avg_weight /= len(x)
    return pd.Series(['L', avg_weight, True], index=['size', 'weight', 'adult'])

expected_df = gb.apply(GrowUp)
expected_df

,size,weight,adult
animal,,,
cat,L,12.4375,True
dog,L,20.0000,True
fish,L,1.2500,True


In [168]:
#### Replacing some values with mean of the rest of group

df = pd.DataFrame({'A' : [1, 1, 2, 2], 'B' : [1, -1, 1, 2]})
df

,A,B
0,1,1
1,1,-1
2,2,1
3,2,2


In [178]:
# Replace negative values as the mean of positive values
gb = df.groupby('A')


def replace(g):
    mask = g<0
    g.loc[mask] = g[~mask].mean()
    return g

gb.transform(replace)


,B
0,1
1,1
2,1
3,2


In [179]:
#### Create multiple aggregated columns
rng = pd.date_range('2015-11-11', periods=10, freq='2min')

ts = Series(data = list(range(10)), index=rng)
ts

2015-11-11 00:00:00    0
2015-11-11 00:02:00    1
2015-11-11 00:04:00    2
2015-11-11 00:06:00    3
2015-11-11 00:08:00    4
2015-11-11 00:10:00    5
2015-11-11 00:12:00    6
2015-11-11 00:14:00    7
2015-11-11 00:16:00    8
2015-11-11 00:18:00    9
Freq: 2T, dtype: int64

In [181]:
def my_cust(x):
    if len(x) > 2:
        return x[1] * 1.234
    return pd.NaT  # NaT means Not A Time; identical to np.nan

mhc = {'Mean': np.mean,
       'Max': np.max,
       'Custom': my_cust}

ts.resample('5min', how=mhc)

,Custom,Max,Mean
2015-11-11 00:00:00,1.234,2,1.0
2015-11-11 00:05:00,NaN,4,3.5
2015-11-11 00:10:00,7.404,7,6.0
2015-11-11 00:15:00,NaN,9,8.5


#### Create a value counts column and reassign back to the DataFrame

In [183]:
df = pd.DataFrame({'Color': 'Red Red Red Blue'.split(),
   .....:                    'Value': [100, 150, 50, 50]}); df
df

,Color,Value
0,Red,100
1,Red,150
2,Red,50
3,Blue,50


In [184]:
df['Counts'] = df.groupby(['Color']).transform(len)
df

,Color,Value,Counts
0,Red,100,3
1,Red,150,3
2,Red,50,3
3,Blue,50,1


#### Shift groups of the values in a column based on the index


In [185]:
df = pd.DataFrame(
   .....:    {u'line_race': [10, 10, 8, 10, 10, 8],
   .....:     u'beyer': [99, 102, 103, 103, 88, 100]},
   .....:     index=[u'Last Gunfighter', u'Last Gunfighter', u'Last Gunfighter',
   .....:            u'Paynter', u'Paynter', u'Paynter']); df
df

,beyer,line_race
Last Gunfighter,99,10
Last Gunfighter,102,10
Last Gunfighter,103,8
Paynter,103,10
Paynter,88,10
Paynter,100,8


In [188]:
df['beyer_shifted'] = df.groupby(level=0)['beyer'].shift(1)
df['beyer_shifted_back'] = df.groupby(level=0)['beyer'].shift(-1)
df

,beyer,line_race,beyer_shifted,beyer_shifted_back
Last Gunfighter,99,10,NaN,102
Last Gunfighter,102,10,99,103
Last Gunfighter,103,8,102,NaN
Paynter,103,10,NaN,88
Paynter,88,10,103,100
Paynter,100,8,88,NaN


#### Select row with maximum value from each group

In [190]:
df = pd.DataFrame({'host':['other','other','that','this','this'],
   .....:                    'service':['mail','web','mail','mail','web'],
   .....:                    'no':[1, 2, 1, 2, 1]}).set_index(['host', 'service'])
df

no
host  service    
other mail      1
      web       2
that  mail      1
this  mail      2
      web       1

In [192]:
# Method 1
df['max_value'] = df.groupby(level=0).transform(np.max)
df

no  max_value
host  service               
other mail      1          2
      web       2          2
that  mail      1          1
this  mail      2          2
      web       1          2

In [196]:
# Method 2
mask = df.groupby(level=0).agg('idxmax')
df_count = df.loc[mask['no']].reset_index()
df_count

,host,service,no,max_value
0,other,web,2,2
1,that,mail,1,1
2,this,mail,2,2


## Rolling computation

In [198]:
data_dict = {'RollBasis': [1, 1, 1, 2, 3, 5, 8, 10, 12, 13],
             'ToRoll': [1, 4, -5, 2, -4, -2, 0, -13, -2, -5]}
df = DataFrame(data_dict)
df

,RollBasis,ToRoll
0,1,1
1,1,4
2,1,-5
3,2,2
4,3,-4
5,5,-2
6,8,0
7,10,-13
8,12,-2
9,13,-5


In [202]:
def f(x):
    ser = df.ToRoll[(df.RollBasis >= x) & (df.RollBasis < x+5)]
    return ser.sum()

df['Rolled'] = df.RollBasis.apply(f)
df

,RollBasis,ToRoll,Rolled
0,1,1,-4
1,1,4,-4
2,1,-5,-4
3,2,2,-4
4,3,-4,-6
5,5,-2,-2
6,8,0,-15
7,10,-13,-20
8,12,-2,-7
9,13,-5,-5


#### Time Series Alignment and To-Date Functions

I have a dataset with the following first three columns. Include Basket ID (unique identifier), Sale amount (dollars) and date of the transaction. I want to calculate the following column for each row of the dataset, and I would like to it in Python.

Previous Sale of the same basket (if any); Sale Count to date for current basket; Mean To Date for current basket (if available); Max To Date for current basket (if available)

In [207]:
data_dict = {'Basket': [88, 88, 88, 123, 477, 477, 566],
             'Sale': [15, 30, 16, 90, 77, 57, 90],
             'Date': pd.date_range('2015-11-11', periods=7, freq='B')}
df = pd.DataFrame(data_dict)
df

,Basket,Date,Sale
0,88,2015-11-11,15
1,88,2015-11-12,30
2,88,2015-11-13,16
3,123,2015-11-16,90
4,477,2015-11-17,77
5,477,2015-11-18,57
6,566,2015-11-19,90


In [211]:
def handler(grouped):
    ts = grouped.set_index('Date')['Sale'].sort_index()
    # ts is the (ordered) time series of sales restricted to a single basket
    # we can now create a dataframe by combining different metrics
    # pandas has a function for each of the ones you are interested in!
    
    return pd.concat(
        {
            'MeanToDate': pd.stats.moments.expanding_mean(ts),  # cumulative mean
            'MaxToDate': ts.cummax(),  # cumulative max
            'SaleCount': pd.stats.moments.expanding_count(ts),  # cumulative count
            'Sale': ts,  # simple copy
            'PrevSale': ts.shift(1)  # previous sale
        }, axis=1
    )

df_new = df.groupby('Basket').apply(handler)
df_new

MaxToDate  MeanToDate  PrevSale  Sale  SaleCount
Basket Date                                                        
88     2015-11-11         15   15.000000       NaN    15          1
       2015-11-12         30   22.500000        15    30          2
       2015-11-13         30   20.333333        30    16          3
123    2015-11-16         90   90.000000       NaN    90          1
477    2015-11-17         77   77.000000       NaN    77          1
       2015-11-18         77   67.000000        77    57          2
566    2015-11-19         90   90.000000       NaN    90          1

In [212]:
df_new.reset_index()

,Basket,Date,MaxToDate,MeanToDate,PrevSale,Sale,SaleCount
0,88,2015-11-11,15,15.000000,NaN,15,1
1,88,2015-11-12,30,22.500000,15,30,2
2,88,2015-11-13,30,20.333333,30,16,3
3,123,2015-11-16,90,90.000000,NaN,90,1
4,477,2015-11-17,77,77.000000,NaN,77,1
5,477,2015-11-18,77,67.000000,77,57,2
6,566,2015-11-19,90,90.000000,NaN,90,1
